In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import math as mt
import warnings

# 한글출력
matplotlib.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False
warnings.filterwarnings(action='ignore') 

from src import crs, PublicPredictor
from src.dbc import utils, TimeDivisionKMeans
from IPython.display import clear_output

In [3]:
# 1. load_excel
data_path = "data/apt_1.xlsx"
xlsx = pd.read_excel(data_path, header=None,
                     skiprows=2, engine="openpyxl")

# 2. data preprocessing
p, m = crs.utils.data_preprocessing(xlsx)

m.set_index("month", inplace=True)

# 3. data init
_month = 1

month_df = pd.DataFrame(m.loc[_month])
month_df.reset_index(inplace=True)

month_df.columns = ['name', 'usage (kWh)']

PUBLIC_PERCENTAGE = 30
APT = crs.utils.get_APT(month_df, PUBLIC_PERCENTAGE)

calc = crs.models.ManagementOffice(
        month=_month,
        households=month_df,
        APT=APT,
        contract="단일계약"
    )
apt = calc.apart

In [4]:
m_15 = utils.data_preprocessing(xlsx)

df = utils.dimension_reduction(m_15)
m_60 = df.copy()

m_15_1 = m_15[m_15.index.month == 1].copy()
m_60_1 = m_60[m_60.index.month == 1].copy()

## KMeans Setting

In [31]:
from sklearn.metrics import euclidean_distances as euc
from src.KMeans import KMeans

ecv_chk = pd.DataFrame(columns=['KMeans', 'KMeans++'])


for case in range(0, 100):
    kmeans_1 = KMeans(datas=m_60_1.T.values, ver=1)
    kmeans_2 = KMeans(datas=m_60_1.T.values, ver=2)

    kmeans_1.fit()
    kmeans_2.fit()

    clear_output(wait=True)

    ecv_chk = ecv_chk.append({
        "KMeans": round(kmeans_1.ecv * 100),
        "KMeans++": round(kmeans_2.ecv * 100)
    }, ignore_index=True)

ecv_chk

,KMeans,KMeans++
0,45,51
1,40,51
2,38,51
3,45,51
4,42,50
...,...,...
95,44,51
96,38,50
97,49,51
98,43,51


In [38]:
ecv_chk['KMeans'].mean()

43.19

In [39]:
ecv_chk['KMeans++'].mean()

50.67

In [24]:
print(euc(kmeans_1.clusters_, kmeans_1.clusters_).mean())
print(euc(kmeans_2.clusters_, kmeans_2.clusters_).mean())

10.599332667838961
15.31524454971364


In [40]:
from sklearn.metrics import euclidean_distances as euc
from src.KMeans import KMeans

outlier_chk = pd.DataFrame(columns=['KMeans', 'KMeans++'])


for case in range(0, 100):
    kmeans_1 = KMeans(datas=m_60_1.T.values, ver=1)
    kmeans_2 = KMeans(datas=m_60_1.T.values, ver=2)

    kmeans_1.fit()
    kmeans_2.fit()

    clear_output(wait=True)

    outlier_chk = outlier_chk.append({
        "KMeans": np.where(pd.Series(kmeans_1.labels_).value_counts().values == 1)[0].size,
        "KMeans++": np.where(pd.Series(kmeans_2.labels_).value_counts().values == 1)[0].size
    }, ignore_index=True)

outlier_chk

,KMeans,KMeans++
0,3,4
1,1,4
2,1,3
3,3,4
4,1,3
...,...,...
95,3,4
96,2,4
97,1,2
98,2,3


In [47]:
outlier_chk['KMeans'].max()

4

In [48]:
outlier_chk['KMeans++'].max()

4

In [ ]:
outlier_chk['KMeans'].min()